In [1]:
import h5py
import numpy as np
from common import write_data, write_from_multiple_jsons
from nexusformat.nexus import nxload, tree

In [7]:
dati = np.loadtxt("2018.11.06_0.xyz")
x, y, z = data[:,0], data[:,1], data[:,2]

In [8]:
def write_data_in_nexus(where, dati, measured):
    data = where.create_group("data")
    data.attrs["NX_class"] = "NXdata"
    data.attrs["signal"] = "z"
    data.attrs["axes"] = ["x", "y"]
    x, y, z = dati[:,0], dati[:,1], dati[:,2]
    data.create_dataset("x", data= x)
    data["x"].attrs["units"]= "m"
    data.create_dataset("y", data= y)
    data["y"].attrs["units"]= "m"
    data.create_dataset("z", data= z)
    if measured == "height":
        data["z"].attrs["units"]= "m"
    elif measured == "phase":
        data["z"].attrs["units"]= "deg"        
    data.create_dataset("title", data= "Title for plot")

In [15]:
with h5py.File("afm_file_auto1.nxs", "w") as f:
    entry = f.create_group("entry")
    entry.attrs["NX_class"] = "NXentry"
    entry.attrs["default"] = "/entry/data"
    entry.create_dataset("definition", data="NXafm")
    entry["definition"].attrs["version"] = "v2024.02"
    entry["definition"].attrs["URL"] = "https://github.com/FAIRmat-NFDI/nexus_definitions/blob/fairmat/contributed_definitions/NXafm.nxdl.xml"
    write_from_multiple_jsons("./Codici e file per afm/jsons_for_afm/" , entry)
    #### Completiamo generando i link necessari
    write_data_in_nexus(entry, dati, "height")
    scan = entry["instrument"].create_group ("scan_environment")
    scan.attrs["NX_class"] = "NXenvironment"
    scan["height_piezo_sensor"] = entry["instrument"]["height_piezo_sensor"]
    scan["XY_piezo_sensor"] = entry["instrument"]["XY_piezo_sensor"]
    scan["tip_temp_sensor"] = entry["instrument"]["tip_temp_sensor"]
    scan.create_dataset("tip_temp", data=25)
    scan["tip_temp"].attrs["units"] = "celsius"
    #### Reproducibility indicators
    reprind = entry.create_group("reproducibility_indicators")
    reprind.attrs["NX_class"] = "NXcollection"
    reprind["cantilever_oscillator"] = entry["instrument"]["cantilever"]["cantilever_oscillator"]
    reprind["cantilever_tip_temp"] = entry["instrument"]["scan_environment"]["tip_temp"]
    #### Resolution indicators
    resind = entry.create_group("resolution_indicators")
    resind.attrs["NX_class"] = "NXcollection"
    resind["oscillator_excitation"] = entry["instrument"]["cantilever"]["cantilever_oscillator"]["oscillator_excitation"]
    resind["cantilever_tip_temp"] = entry["instrument"]["scan_environment"]["tip_temp"]
    resind["amplitude_excitation"] = entry["instrument"]["cantilever"]["cantilever_oscillator"]["phase_lock_loop"]["amplitude_excitation"]

In [16]:
from nexusformat.nexus import *

test = nxload("afm_file_auto1.nxs")
print(test.tree);

root:NXroot
  entry:NXentry
    @default = '/entry/data'
    data:NXdata
      @axes = ['x', 'y']
      @signal = 'z'
      title = 'Title for plot'
      x = float64(65536)
        @units = 'm'
      y = float64(65536)
        @units = 'm'
      z = float64(65536)
        @units = 'm'
    definition = 'NXafm'
      @URL = 'https://github.com/FAIRmat-NFDI/nexus_definiti...'
      @version = 'v2024.02'
    experiment_facility = 'MNF'
    experiment_identifier = 'IDAFM1'
    experiment_institution = 'FBK'
    experiment_laboratory = 'CRD'
    experiment_location = 'Trento'
    instrument:NXinstrument
      XY_piezo_sensor:NXsensor
        attached_to = 'chuck'
        measurement = 'current'
        model = 'model 2'
        name = 'nome sensore'
        piezo_configuration:NXpiezo_config_spm
          calibration:NXcalibration
            calibration_name = 'Calibrazione 11/08/2025'
            calibration_type = 'passive'
          piezo_material:NXpiezoelectric_material
            ch